In [2]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [5]:
from transformers import pipeline

# Load a sentiment classifier
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [7]:
texts = [
    "This brand is amazing! Love working together.",
    "Worst experience ever. Don't recommend.",
    "Can we discuss rates for this campaign?"
]

sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest"
)



config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


In [8]:
test_texts = [
    # Clear positive
    "This campaign was a huge success and I loved working with the brand.",
    "Great service and super fast support. Highly recommended!",
    "So grateful for this collaboration opportunity!",
    # Clear negative
    "I’m extremely disappointed with the final results.",
    "Worst brand collab I’ve ever done.",
    "The partner was rude and unprofessional.",
    # Neutral / Business-like
    "Can you share the campaign brief?",
    "Let's discuss next steps for our collaboration.",
    "The payment terms are acceptable.",
    "I have received the product. Thank you.",
    "Looking forward to your reply.",
    # Edge/uncertain (sarcasm/faint praise)
    "Well, that went better than expected... not!",
    "I guess this is okay.",
    "It's fine, nothing special.",
    # Strong emotion (positive/negative)
    "Absolutely LOVE this product!",
    "I will never work with them again, period.",
    # Spam/repetitive (to test robustness)
    "Buy followers now! Get 10k for just $10!",
    "Check out my new post!!!"
]


In [9]:
results = sentiment_analyzer(test_texts)
for text, result in zip(test_texts, results):
    print(f"Text: {text}")
    print(f"Sentiment: {result['label']}, Score: {result['score']:.2f}\n")


Text: This campaign was a huge success and I loved working with the brand.
Sentiment: positive, Score: 0.99

Text: Great service and super fast support. Highly recommended!
Sentiment: positive, Score: 0.98

Text: So grateful for this collaboration opportunity!
Sentiment: positive, Score: 0.98

Text: I’m extremely disappointed with the final results.
Sentiment: negative, Score: 0.93

Text: Worst brand collab I’ve ever done.
Sentiment: negative, Score: 0.95

Text: The partner was rude and unprofessional.
Sentiment: negative, Score: 0.88

Text: Can you share the campaign brief?
Sentiment: neutral, Score: 0.93

Text: Let's discuss next steps for our collaboration.
Sentiment: neutral, Score: 0.60

Text: The payment terms are acceptable.
Sentiment: positive, Score: 0.55

Text: I have received the product. Thank you.
Sentiment: positive, Score: 0.81

Text: Looking forward to your reply.
Sentiment: positive, Score: 0.93

Text: Well, that went better than expected... not!
Sentiment: positive, S

In [10]:
# --- INTERACTIVE SENTIMENT ANALYSIS SYSTEM ---

from transformers import pipeline
import time

class InteractiveSentimentAnalyzer:
    def __init__(self):
        print("🤖 Loading sentiment analysis models...")

        # Load different sentiment models
        self.models = {
            "basic": {
                "name": "Basic (Positive/Negative)",
                "pipeline": pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english"),
                "labels": {"POSITIVE": "🟢 Positive", "NEGATIVE": "🔴 Negative"}
            },
            "advanced": {
                "name": "Advanced (Positive/Neutral/Negative)",
                "pipeline": pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest"),
                "labels": {"LABEL_0": "🔴 Negative", "LABEL_1": "🟡 Neutral", "LABEL_2": "🟢 Positive"}
            },
            "emotion": {
                "name": "Emotion Detection",
                "pipeline": pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base"),
                "labels": {}  # Will be dynamic based on output
            }
        }

        self.current_model = "basic"
        print("✅ Models loaded successfully!")

    def classify_sentiment(self, text, model_type="basic"):
        """
        Classify sentiment of input text
        """
        try:
            model_info = self.models[model_type]
            result = model_info["pipeline"](text)[0]

            # Format the output
            label = result['label']
            score = result['score']
            confidence = score * 100

            # Get display label
            if model_type == "emotion":
                display_label = f"😊 {label.title()}"
            else:
                display_label = model_info["labels"].get(label, label)

            return {
                "text": text,
                "sentiment": display_label,
                "raw_label": label,
                "confidence": confidence,
                "model_used": model_info["name"]
            }

        except Exception as e:
            return {
                "error": f"Classification failed: {str(e)}",
                "text": text
            }

    def display_result(self, result):
        """
        Display classification result in a nice format
        """
        if "error" in result:
            print(f"❌ {result['error']}")
            return

        print(f"\n📝 Text: '{result['text']}'")
        print(f"🎯 Sentiment: {result['sentiment']}")
        print(f"📊 Confidence: {result['confidence']:.1f}%")
        print(f"🤖 Model: {result['model_used']}")
        print("-" * 50)

def run_interactive_classifier():
    """
    Main interactive sentiment classification system
    """
    print("🎯 INTERACTIVE SENTIMENT ANALYSIS SYSTEM")
    print("=" * 50)
    print("Enter text to analyze sentiment in real-time!")
    print("Commands:")
    print("  'quit' - Exit the system")
    print("  'switch' - Change model type")
    print("  'help' - Show available commands")
    print("-" * 50)

    analyzer = InteractiveSentimentAnalyzer()

    while True:
        print(f"\nCurrent model: {analyzer.models[analyzer.current_model]['name']}")
        user_input = input("Enter text to analyze: ").strip()

        if user_input.lower() == 'quit':
            print("👋 Thanks for using the sentiment analyzer!")
            break

        elif user_input.lower() == 'switch':
            print("\nAvailable models:")
            for key, model_info in analyzer.models.items():
                current = " (current)" if key == analyzer.current_model else ""
                print(f"  {key}: {model_info['name']}{current}")

            choice = input("Select model (basic/advanced/emotion): ").strip().lower()
            if choice in analyzer.models:
                analyzer.current_model = choice
                print(f"✅ Switched to {analyzer.models[choice]['name']}")
            else:
                print("❌ Invalid model choice")
            continue

        elif user_input.lower() == 'help':
            print("\n📚 HELP:")
            print("  • Type any text to get sentiment analysis")
            print("  • 'basic' model: Simple positive/negative classification")
            print("  • 'advanced' model: Positive/neutral/negative classification")
            print("  • 'emotion' model: Detects specific emotions (joy, anger, etc.)")
            print("  • Higher confidence scores indicate more certain predictions")
            continue

        elif not user_input:
            print("⚠️ Please enter some text to analyze")
            continue

        # Analyze the sentiment
        print("🔄 Analyzing...")
        start_time = time.time()

        result = analyzer.classify_sentiment(user_input, analyzer.current_model)

        end_time = time.time()
        processing_time = (end_time - start_time) * 1000

        # Display results
        analyzer.display_result(result)
        print(f"⚡ Processing time: {processing_time:.0f}ms")

# Run the interactive system
run_interactive_classifier()


🎯 INTERACTIVE SENTIMENT ANALYSIS SYSTEM
Enter text to analyze sentiment in real-time!
Commands:
  'quit' - Exit the system
  'switch' - Change model type
  'help' - Show available commands
--------------------------------------------------
🤖 Loading sentiment analysis models...


Device set to use cpu
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


✅ Models loaded successfully!

Current model: Basic (Positive/Negative)
Enter text to analyze: i do not  like you
🔄 Analyzing...

📝 Text: 'i do not  like you'
🎯 Sentiment: 🔴 Negative
📊 Confidence: 99.9%
🤖 Model: Basic (Positive/Negative)
--------------------------------------------------
⚡ Processing time: 1252ms

Current model: Basic (Positive/Negative)
Enter text to analyze: this campaign is hit
🔄 Analyzing...

📝 Text: 'this campaign is hit'
🎯 Sentiment: 🟢 Positive
📊 Confidence: 99.9%
🤖 Model: Basic (Positive/Negative)
--------------------------------------------------
⚡ Processing time: 125ms

Current model: Basic (Positive/Negative)
Enter text to analyze: lets lower your price offer
🔄 Analyzing...

📝 Text: 'lets lower your price offer'
🎯 Sentiment: 🔴 Negative
📊 Confidence: 99.9%
🤖 Model: Basic (Positive/Negative)
--------------------------------------------------
⚡ Processing time: 74ms

Current model: Basic (Positive/Negative)
Enter text to analyze: you wont be able to afford me
🔄

In [11]:
# --- COMPREHENSIVE EMOTION DETECTION SYSTEM ---

from transformers import pipeline
import numpy as np

class EmotionDetectionSystem:
    def __init__(self):
        print("🧠 Loading emotion detection models...")

        # Different emotion detection models
        self.models = {
            "emotions_6": {
                "name": "6 Basic Emotions",
                "pipeline": pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base"),
                "emotions": ["sadness", "joy", "love", "anger", "fear", "surprise"]
            },
            "emotions_27": {
                "name": "27 Detailed Emotions",
                "pipeline": pipeline("text-classification", model="SamLowe/roberta-base-go_emotions"),
                "emotions": [
                    "admiration", "amusement", "anger", "annoyance", "approval", "caring",
                    "confusion", "curiosity", "desire", "disappointment", "disapproval",
                    "disgust", "embarrassment", "excitement", "fear", "gratitude",
                    "grief", "joy", "love", "nervousness", "optimism", "pride",
                    "realization", "relief", "remorse", "sadness", "surprise"
                ]
            },
            "emotions_custom": {
                "name": "Business/Social Emotions",
                "pipeline": pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-emotion"),
                "emotions": ["anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism", "sadness", "surprise", "trust"]
            }
        }

        self.current_model = "emotions_6"
        print("✅ Emotion detection models loaded!")

    def detect_emotions(self, text, model_type="emotions_6", top_k=5):
        """
        Detect emotions in text with confidence scores
        """
        try:
            model_info = self.models[model_type]
            results = model_info["pipeline"](text, return_all_scores=True)[0]

            # Sort by confidence score
            sorted_results = sorted(results, key=lambda x: x['score'], reverse=True)

            # Get top emotions
            top_emotions = sorted_results[:top_k]

            return {
                "text": text,
                "primary_emotion": sorted_results[0]['label'],
                "primary_confidence": sorted_results[0]['score'] * 100,
                "all_emotions": [
                    {
                        "emotion": result['label'],
                        "confidence": result['score'] * 100,
                        "emoji": self.get_emotion_emoji(result['label'])
                    }
                    for result in top_emotions
                ],
                "model_used": model_info["name"]
            }

        except Exception as e:
            return {"error": f"Emotion detection failed: {str(e)}", "text": text}

    def get_emotion_emoji(self, emotion):
        """Get emoji representation for emotions"""
        emotion_emojis = {
            # Basic emotions
            "joy": "😊", "happiness": "😊", "love": "❤️", "anger": "😠",
            "fear": "😨", "sadness": "😢", "surprise": "😮", "disgust": "🤢",

            # Detailed emotions
            "admiration": "🤩", "amusement": "😄", "annoyance": "😤",
            "approval": "👍", "caring": "🤗", "confusion": "😕",
            "curiosity": "🤔", "desire": "😍", "disappointment": "😞",
            "disapproval": "👎", "embarrassment": "😳", "excitement": "🤩",
            "gratitude": "🙏", "grief": "😭", "nervousness": "😰",
            "optimism": "🌟", "pride": "😤", "realization": "💡",
            "relief": "😌", "remorse": "😔", "trust": "🤝",

            # Business emotions
            "anticipation": "⏳", "pessimism": "😞", "betrayed": "💔",
            "frustrated": "😫", "satisfied": "😌", "anxious": "😟"
        }
        return emotion_emojis.get(emotion.lower(), "🎭")

    def display_emotion_results(self, result):
        """Display emotion detection results"""
        if "error" in result:
            print(f"❌ {result['error']}")
            return

        print(f"\n📝 Text: '{result['text']}'")
        print(f"🎭 Primary Emotion: {result['all_emotions'][0]['emoji']} {result['primary_emotion'].title()} ({result['primary_confidence']:.1f}%)")
        print(f"🤖 Model: {result['model_used']}")

        print(f"\n📊 All Detected Emotions:")
        for emotion_data in result['all_emotions']:
            confidence_bar = "█" * int(emotion_data['confidence'] / 10)
            print(f"  {emotion_data['emoji']} {emotion_data['emotion'].title():<15} {confidence_bar} {emotion_data['confidence']:.1f}%")

        print("-" * 60)

# Initialize emotion detection system
emotion_detector = EmotionDetectionSystem()


🧠 Loading emotion detection models...


Device set to use cpu


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cpu


✅ Emotion detection models loaded!


In [12]:
# --- INTERACTIVE EMOTION DETECTION ---

def run_emotion_analysis():
    """
    Interactive emotion detection system
    """
    print("🎭 EMOTION DETECTION SYSTEM")
    print("=" * 40)
    print("Detect detailed emotions like anger, joy, fear, love, surprise, and many more!")
    print("Available models:")
    print("  1. Basic (6 emotions): joy, anger, fear, sadness, surprise, love")
    print("  2. Detailed (27 emotions): includes frustration, disappointment, excitement, etc.")
    print("  3. Social (11 emotions): optimized for social media and business communications")
    print("-" * 60)

    detector = EmotionDetectionSystem()

    while True:
        print(f"\nCurrent model: {detector.models[detector.current_model]['name']}")
        user_input = input("Enter text to analyze emotions (or 'quit'/'switch'): ").strip()

        if user_input.lower() == 'quit':
            print("👋 Thanks for using emotion detection!")
            break

        elif user_input.lower() == 'switch':
            print("\nAvailable models:")
            for key, model_info in detector.models.items():
                current = " (current)" if key == detector.current_model else ""
                print(f"  {key}: {model_info['name']}{current}")
                print(f"    Emotions: {', '.join(model_info['emotions'][:5])}...")

            choice = input("Select model (emotions_6/emotions_27/emotions_custom): ").strip()
            if choice in detector.models:
                detector.current_model = choice
                print(f"✅ Switched to {detector.models[choice]['name']}")
            else:
                print("❌ Invalid model choice")
            continue

        elif not user_input:
            print("⚠️ Please enter some text to analyze")
            continue

        # Detect emotions
        print("🔄 Analyzing emotions...")
        result = detector.detect_emotions(user_input, detector.current_model, top_k=8)
        detector.display_emotion_results(result)

# Start emotion analysis
run_emotion_analysis()


🎭 EMOTION DETECTION SYSTEM
Detect detailed emotions like anger, joy, fear, love, surprise, and many more!
Available models:
  1. Basic (6 emotions): joy, anger, fear, sadness, surprise, love
  2. Detailed (27 emotions): includes frustration, disappointment, excitement, etc.
  3. Social (11 emotions): optimized for social media and business communications
------------------------------------------------------------
🧠 Loading emotion detection models...


Device set to use cpu
Device set to use cpu
Device set to use cpu


✅ Emotion detection models loaded!

Current model: 6 Basic Emotions
Enter text to analyze emotions (or 'quit'/'switch'): i love you


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


🔄 Analyzing emotions...

📝 Text: 'i love you'
🎭 Primary Emotion: 😊 Joy (96.4%)
🤖 Model: 6 Basic Emotions

📊 All Detected Emotions:
  😊 Joy             █████████ 96.4%
  😢 Sadness          2.8%
  😮 Surprise         0.4%
  😠 Anger            0.3%
  🎭 Neutral          0.1%
  😨 Fear             0.1%
  🤢 Disgust          0.0%
------------------------------------------------------------

Current model: 6 Basic Emotions
Enter text to analyze emotions (or 'quit'/'switch'): i hate you
🔄 Analyzing emotions...

📝 Text: 'i hate you'
🎭 Primary Emotion: 😢 Sadness (52.5%)
🤖 Model: 6 Basic Emotions

📊 All Detected Emotions:
  😢 Sadness         █████ 52.5%
  😠 Anger           ████ 44.6%
  😊 Joy              1.2%
  🤢 Disgust          0.8%
  😮 Surprise         0.4%
  🎭 Neutral          0.3%
  😨 Fear             0.2%
------------------------------------------------------------

Current model: 6 Basic Emotions
Enter text to analyze emotions (or 'quit'/'switch'): i will kill you
🔄 Analyzing emotions...

📝 T

In [13]:
# --- EMOTION TEST CASES ---

def test_emotion_detection():
    """
    Test emotion detection with various examples
    """
    detector = EmotionDetectionSystem()

    test_cases = {
        "Anger/Frustration": [
            "I can't believe they canceled our collaboration last minute!",
            "This is absolutely ridiculous and unprofessional!",
            "I'm so frustrated with their lack of communication."
        ],
        "Joy/Excitement": [
            "I'm thrilled about this new partnership opportunity!",
            "This campaign exceeded all my expectations!",
            "Can't wait to share this amazing product with my audience!"
        ],
        "Fear/Anxiety": [
            "I'm worried this collaboration might not work out.",
            "What if my audience doesn't like this sponsored content?",
            "I'm nervous about the campaign results."
        ],
        "Sadness/Disappointment": [
            "I'm disappointed with how the campaign turned out.",
            "This collaboration didn't meet my expectations.",
            "I feel let down by the brand's response."
        ],
        "Love/Admiration": [
            "I absolutely adore working with this brand!",
            "This company truly cares about their influencer partners.",
            "I'm grateful for this incredible opportunity."
        ],
        "Surprise/Confusion": [
            "I wasn't expecting such amazing results!",
            "This collaboration outcome surprised me completely.",
            "I'm confused about the campaign requirements."
        ],
        "Complex/Mixed Emotions": [
            "I'm excited but also nervous about this big campaign.",
            "While I appreciate the opportunity, I'm concerned about the timeline.",
            "This partnership has been both rewarding and challenging."
        ]
    }

    print("🧪 COMPREHENSIVE EMOTION TESTING")
    print("=" * 50)

    for category, texts in test_cases.items():
        print(f"\n📂 {category.upper()}")
        print("-" * 30)

        for text in texts:
            result = detector.detect_emotions(text, "emotions_27", top_k=3)
            if "error" not in result:
                primary = result['all_emotions'][0]
                print(f"Text: '{text}'")
                print(f"Primary: {primary['emoji']} {primary['emotion'].title()} ({primary['confidence']:.1f}%)")

                # Show secondary emotions
                if len(result['all_emotions']) > 1:
                    secondary = [f"{e['emoji']} {e['emotion']}" for e in result['all_emotions'][1:3]]
                    print(f"Also detected: {', '.join(secondary)}")
                print()

# Run comprehensive testing
test_emotion_detection()


🧠 Loading emotion detection models...


Device set to use cpu
Device set to use cpu
Device set to use cpu


✅ Emotion detection models loaded!
🧪 COMPREHENSIVE EMOTION TESTING

📂 ANGER/FRUSTRATION
------------------------------
Text: 'I can't believe they canceled our collaboration last minute!'
Primary: 😮 Surprise (63.9%)
Also detected: 🎭 neutral, 😞 disappointment

Text: 'This is absolutely ridiculous and unprofessional!'
Primary: 😤 Annoyance (56.6%)
Also detected: 😠 anger, 🤢 disgust

Text: 'I'm so frustrated with their lack of communication.'
Primary: 😤 Annoyance (56.7%)
Also detected: 😠 anger, 😞 disappointment


📂 JOY/EXCITEMENT
------------------------------
Text: 'I'm thrilled about this new partnership opportunity!'
Primary: 🤩 Excitement (79.5%)
Also detected: 😊 joy, 🎭 neutral

Text: 'This campaign exceeded all my expectations!'
Primary: 🤩 Admiration (29.3%)
Also detected: 🤩 excitement, 😮 surprise

Text: 'Can't wait to share this amazing product with my audience!'
Primary: 🤩 Excitement (74.2%)
Also detected: 🤔 curiosity, 🤩 admiration


📂 FEAR/ANXIETY
------------------------------
Text:

In [23]:
# --- COMPLETE FIXED EMOTION DETECTION MODEL SAVING CODE ---

import os
import json
import pickle
from datetime import datetime
from transformers import pipeline

# Define emotion detection system class globally
class EmotionDetectionSystem:
    def __init__(self):
        print("🧠 Loading emotion detection models for saving...")

        self.models = {
            "emotions_6": {
                "name": "6 Basic Emotions",
                "model_path": "j-hartmann/emotion-english-distilroberta-base",
                "emotions": ["sadness", "joy", "love", "anger", "fear", "surprise"]
            },
            "emotions_27": {
                "name": "27 Detailed Emotions",
                "model_path": "SamLowe/roberta-base-go_emotions",
                "emotions": [
                    "admiration", "amusement", "anger", "annoyance", "approval", "caring",
                    "confusion", "curiosity", "desire", "disappointment", "disapproval",
                    "disgust", "embarrassment", "excitement", "fear", "gratitude",
                    "grief", "joy", "love", "nervousness", "optimism", "pride",
                    "realization", "relief", "remorse", "sadness", "surprise"
                ]
            },
            "emotions_social": {
                "name": "Social Media Emotions",
                "model_path": "cardiffnlp/twitter-roberta-base-emotion",
                "emotions": ["anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism", "sadness", "surprise", "trust"]
            }
        }

        # Load and save each model
        self.loaded_models = {}
        for model_key, model_info in self.models.items():
            try:
                print(f"📥 Loading {model_info['name']}...")
                pipeline_model = pipeline("text-classification", model=model_info['model_path'])

                # Save the model
                model_save_path = f"{model_dir}/models/{model_key}"
                pipeline_model.save_pretrained(model_save_path)

                self.loaded_models[model_key] = {
                    "pipeline": pipeline_model,
                    "info": model_info,
                    "save_path": model_save_path
                }
                print(f"✅ {model_info['name']} saved successfully")

            except Exception as e:
                print(f"❌ Error saving {model_info['name']}: {e}")


    def get_emotion_emoji(self, emotion):
        """Get emoji representation for emotions"""
        emotion_emojis = {
            # Basic emotions
            "joy": "😊", "happiness": "😊", "love": "❤️", "anger": "😠",
            "fear": "😨", "sadness": "😢", "surprise": "😮", "disgust": "🤢",

            # Detailed emotions
            "admiration": "🤩", "amusement": "😄", "annoyance": "😤",
            "approval": "👍", "caring": "🤗", "confusion": "😕",
            "curiosity": "🤔", "desire": "😍", "disappointment": "😞",
            "disapproval": "👎", "disgust": "🤢", "embarrassment": "😳", "excitement": "🤩",
            "gratitude": "🙏", "grief": "😭", "joy": "😊", "love": "❤️", "nervousness": "😰",
            "optimism": "🌟", "pride": "😤", "realization": "💡",
            "relief": "😌", "remorse": "😔", "sadness": "😢", "surprise": "😮", "trust": "🤝",

            # Social emotions
            "anticipation": "⏳", "pessimism": "😞", "betrayed": "💔",
            "frustrated": "😫", "satisfied": "😌", "anxious": "😟"
        }
        return emotion_emojis.get(emotion.lower(), "🎭")

    def detect_emotions(self, text, model_type="emotions_6", top_k=5):
        """Detect emotions in text"""
        try:
            if model_type not in self.loaded_models:
                return {"error": f"Model {model_type} not available"}

            pipeline_model = self.loaded_models[model_type]["pipeline"]
            results = pipeline_model(text, return_all_scores=True)[0]

            # Sort by confidence
            sorted_results = sorted(results, key=lambda x: x['score'], reverse=True)
            top_emotions = sorted_results[:top_k]

            return {
                "text": text,
                "primary_emotion": sorted_results[0]['label'],
                "primary_confidence": sorted_results[0]['score'] * 100,
                "all_emotions": [
                    {
                        "emotion": result['label'],
                        "confidence": result['score'] * 100,
                        "emoji": self.get_emotion_emoji(result['label'])
                    }
                    for result in top_emotions
                ],
                "model_used": self.loaded_models[model_type]["info"]["name"]
            }

        except Exception as e:
            return {"error": f"Emotion detection failed: {str(e)}"}


def save_emotion_detection_model_complete():
    """
    Complete fixed version - save emotion detection model system
    """
    print("💾 SAVING EMOTION DETECTION MODEL SYSTEM")
    print("=" * 50)

    # Create model directory
    model_dir = "emotion_detection_model_v1"
    os.makedirs(model_dir, exist_ok=True)
    os.makedirs(f"{model_dir}/models", exist_ok=True)
    os.makedirs(f"{model_dir}/config", exist_ok=True)

    print(f"📁 Created directory: {model_dir}")

    # Initialize and save the system
    emotion_system = EmotionDetectionSystem()

    # Save the complete system as pickle
    print("🥒 Saving complete emotion system...")
    with open(f"{model_dir}/emotion_system.pkl", 'wb') as f:
        pickle.dump(emotion_system, f)

    # Save configuration and metadata
    print("📝 Saving configuration...")
    config = {
        "model_name": "Emotion Detection System",
        "version": "1.0",
        "created_date": datetime.now().isoformat(),
        "models": {
            model_key: {
                "name": model_data["info"]["name"],
                "emotions_count": len(model_data["info"]["emotions"]),
                "emotions": model_data["info"]["emotions"],
                "save_path": model_data["save_path"]
            }
            for model_key, model_data in emotion_system.loaded_models.items()
        },
        "capabilities": [
            "6 basic emotions (joy, anger, fear, sadness, surprise, love)",
            "27 detailed emotions (including disappointment, excitement, gratitude)",
            "Social media optimized emotions",
            "Confidence scoring for each emotion",
            "Emoji representation for emotions",
            "Top-k emotion ranking"
        ],
        "usage": {
            "basic_emotions": "detect_emotions(text, 'emotions_6')",
            "detailed_emotions": "detect_emotions(text, 'emotions_27')",
            "social_emotions": "detect_emotions(text, 'emotions_social')"
        }
    }

    with open(f"{model_dir}/config/model_config.json", 'w') as f:
        json.dump(config, f, indent=2)

    # Create loader script
    print("🔧 Creating loader script...")
    loader_code = """# --- EMOTION DETECTION MODEL LOADER ---

import pickle
import json
from transformers import pipeline
import os

# Define emotion detection system class (must match the saved structure)
class EmotionDetectionSystem:
    def __init__(self):
        # This __init__ is for loading purposes; models are loaded from disk
        self.loaded_models = {}
        self.models = {} # Placeholder, will be filled from config

    def load_models_from_disk(self, model_dir, config):
        print("🔄 Loading models from disk...")
        self.models = {k: v for k, v in config['models'].items()} # Populate models info from config

        for model_key, model_info in self.models.items():
            try:
                print(f"📥 Loading {model_info['name']} from {model_info['save_path']}...")
                # Load the pipeline directly from the saved directory
                pipeline_model = pipeline("text-classification", model=model_info['save_path'])

                self.loaded_models[model_key] = {
                    "pipeline": pipeline_model,
                    "info": model_info,
                    "save_path": model_info['save_path']
                }
                print(f"✅ {model_info['name']} loaded successfully")

            except Exception as e:
                print(f"❌ Error loading {model_info['name']}: {e}")


    def get_emotion_emoji(self, emotion):
        # This method should also be included for the loaded object to work
        emotion_emojis = {
            # Basic emotions
            "joy": "😊", "happiness": "😊", "love": "❤️", "anger": "😠",
            "fear": "😨", "sadness": "😢", "surprise": "😮", "disgust": "🤢",

            # Detailed emotions
            "admiration": "🤩", "amusement": "😄", "annoyance": "😤",
            "approval": "👍", "caring": "🤗", "confusion": "😕",
            "curiosity": "🤔", "desire": "😍", "disappointment": "😞",
            "disapproval": "👎", "disgust": "🤢", "embarrassment": "😳", "excitement": "🤩",
            "gratitude": "🙏", "grief": "😭", "joy": "😊", "love": "❤️", "nervousness": "😰",
            "optimism": "🌟", "pride": "😤", "realization": "💡",
            "relief": "😌", "remorse": "😔", "sadness": "😢", "surprise": "😮", "trust": "🤝",

            # Social emotions
            "anticipation": "⏳", "pessimism": "😞", "betrayed": "💔",
            "frustrated": "😫", "satisfied": "😌", "anxious": "😟"
        }
        return emotion_emojis.get(emotion.lower(), "🎭")

    def detect_emotions(self, text, model_type="emotions_6", top_k=5):
        \"\"\"Detect emotions in text\"\"\"
        try:
            if model_type not in self.loaded_models:
                return {"error": f"Model {model_type} not available"}

            pipeline_model = self.loaded_models[model_type]["pipeline"]
            # Ensure return_all_scores is handled correctly by pipeline if needed
            results = pipeline_model(text, top_k=None if top_k is None else top_k) # Use top_k=None for all scores

            # If top_k is None, results will be a list of lists, flatten it
            if top_k is None and isinstance(results, list) and all(isinstance(i, list) for i in results):
                 results = results[0] # Assuming single text input

            # Ensure results is a list of dicts
            if not isinstance(results, list):
                 results = [results]

            # Sort by confidence
            sorted_results = sorted(results, key=lambda x: x['score'], reverse=True)
            # Apply top_k after sorting if top_k is not None
            if top_k is not None:
                 sorted_results = sorted_results[:top_k]


            return {
                "text": text,
                "primary_emotion": sorted_results[0]['label'] if sorted_results else None,
                "primary_confidence": sorted_results[0]['score'] * 100 if sorted_results else 0,
                "all_emotions": [
                    {
                        "emotion": result['label'],
                        "confidence": result['score'] * 100,
                        "emoji": self.get_emotion_emoji(result['label'])
                    }
                    for result in sorted_results
                ],
                "model_used": self.loaded_models[model_type]["info"]["name"]
            }

        except Exception as e:
            return {"error": f"Emotion detection failed: {str(e)}", "text": text}


def load_emotion_detection_model():
    \"\"\"
    Load the saved emotion detection model system
    \"\"\"
    print("📥 Loading emotion detection model...")

    model_dir = "emotion_detection_model_v1"

    try:
        # Load configuration first
        config_path = os.path.join(model_dir, "config", "model_config.json")
        with open(config_path, 'r') as f:
            config = json.load(f)
        print("✅ Configuration loaded.")

        # Load the pickled system object (without pipelines loaded)
        system_pkl_path = os.path.join(model_dir, "emotion_system.pkl")
        with open(system_pkl_path, 'rb') as f:
            emotion_system = pickle.load(f)
        print("✅ System structure loaded.")

        # Now load the actual pipelines from the saved paths using the config
        emotion_system.load_models_from_disk(model_dir, config)


        print("✅ Emotion detection model loaded successfully!")
        print(f"📊 Available models: {', '.join(config['models'].keys())}")

        return emotion_system, config

    except Exception as e:
        print(f"❌ Error loading model: {e}")
        return None, None

def quick_emotion_test(emotion_system):
    \"\"\"
    Quick test of the loaded model
    \"\"\"
    if emotion_system is None:
        print("❌ Model not loaded")
        return

    test_texts = [
        "I'm absolutely thrilled about this opportunity!",
        "I feel completely betrayed and disappointed.",
        "This situation makes me really anxious and worried."
    ]

    print("\\n🧪 QUICK EMOTION TEST:")
    print("-" * 40)

    for text in test_texts:
        result = emotion_system.detect_emotions(text, "emotions_27", top_k=3)
        if "error" not in result:
            primary = result['all_emotions'][0]
            print(f"Text: '{text}'")
            print(f"Primary: {primary['emoji']} {primary['emotion'].title()} ({primary['confidence']:.1f}%)")

            # Show top 3 emotions
            top_3 = [f"{e['emoji']} {e['emotion']}" for e in result['all_emotions'][:3]]
            print(f"Top emotions: {', '.join(top_3)}")
            print()
        else:
            print(f"❌ Error analyzing '{text}': {result['error']}")

# Usage:
# emotion_system, config = load_emotion_detection_model()
# quick_emotion_test(emotion_system)
"""

    with open(f"{model_dir}/load_model.py", 'w') as f:
        f.write(loader_code)

    # Create API wrapper
    print("🚀 Creating API wrapper...")
    api_code = """# --- EMOTION DETECTION API ---

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import sys
import os

# Add model directory to path
sys.path.append(os.path.dirname(os.path.abspath(__file__)))
from load_model import load_emotion_detection_model

app = FastAPI(title="Emotion Detection API", version="1.0")

# Load model on startup
emotion_system, config = load_emotion_detection_model()

class EmotionRequest(BaseModel):
    text: str
    model_type: str = "emotions_27"
    top_k: int = 5

class EmotionResponse(BaseModel):
    text: str
    primary_emotion: str or None
    primary_confidence: float
    all_emotions: list
    model_used: str or None

@app.post("/detect-emotions", response_model=EmotionResponse)
async def detect_emotions(request: EmotionRequest):
    \"\"\"
    Detect emotions in text
    \"\"\"
    if emotion_system is None:
        raise HTTPException(status_code=500, detail="Model not loaded")

    result = emotion_system.detect_emotions(
        request.text,
        request.model_type,
        request.top_k
    )

    if "error" in result:
        raise HTTPException(status_code=400, detail=result["error"])

    # Ensure primary_emotion and model_used are not None if all_emotions is empty
    primary_emotion = result.get("primary_emotion")
    primary_confidence = result.get("primary_confidence", 0.0)
    model_used = result.get("model_used")
    all_emotions = result.get("all_emotions", [])

    return EmotionResponse(
        text=result.get("text", ""),
        primary_emotion=primary_emotion,
        primary_confidence=primary_confidence,
        all_emotions=all_emotions,
        model_used=model_used
    )


@app.get("/models")
async def get_available_models():
    \"\"\"
    Get list of available emotion models
    \"\"\"
    if config is None:
        raise HTTPException(status_code=500, detail="Configuration not loaded")

    return {
        "available_models": list(config["models"].keys()),
        "model_details": config["models"]
    }

@app.get("/emotions/{model_type}")
async def get_emotions_list(model_type: str):
    \"\"\"
    Get list of emotions for a specific model
    \"\"\"
    if config is None:
        raise HTTPException(status_code=500, detail="Configuration not loaded")

    if model_type not in config["models"]:
        raise HTTPException(status_code=404, detail="Model not found")

    return {
        "model": model_type,
        "emotions": config["models"][model_type]["emotions"],
        "count": config["models"][model_type]["emotions_count"]
    }

# This block is for running the API directly (e.g., in a Colab notebook or script)
# It should not be included if using a separate uvicorn command to run the api.py file
# if __name__ == "__main__":
#     import uvicorn
#     uvicorn.run(app, host="0.0.0.0", port=8000)

"""

    with open(f"{model_dir}/api.py", 'w') as f:
        f.write(api_code)

    # Create README (FIXED - using string concatenation)
    print("📚 Creating documentation...")
    readme_parts = [
        "# 🎭 Emotion Detection Model System\n\n",
        f"**Saved on**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n",
        "## 🎯 Model Capabilities\n\n",
        "This comprehensive emotion detection system can identify **27+ different emotions** including:\n\n",
        "### 6 Basic Emotions\n",
        "- 😊 **Joy** - Happiness, delight, pleasure\n",
        "- 😠 **Anger** - Rage, fury, irritation\n",
        "- 😨 **Fear** - Anxiety, worry, terror\n",
        "- 😢 **Sadness** - Sorrow, grief, melancholy\n",
        "- 😮 **Surprise** - Astonishment, shock, amazement\n",
        "- ❤️ **Love** - Affection, care, adoration\n\n",
        "### 27 Detailed Emotions\n",
        "Includes: admiration, amusement, annoyance, approval, caring, confusion, curiosity, desire, ",
        "disappointment, disapproval, disgust, embarrassment, excitement, gratitude, grief, ",
        "nervousness, optimism, pride, realization, relief, remorse, and more.\n\n",
        "### Social Media Emotions\n",
        "Optimized for: anger, anticipation, disgust, fear, joy, love, optimism, pessimism, sadness, surprise, trust\n\n",
        "## 🚀 Quick Start\n\n",

        "```python\n",
        "# Load the model\n",
        "from emotion_detection_model_v1.load_model import load_emotion_detection_model, quick_emotion_test\n\n",
        "emotion_system, config = load_emotion_detection_model()\n\n",
        "# Test the model\n",
        "quick_emotion_test(emotion_system)\n\n",
        "# Use the model\n",
        "result = emotion_system.detect_emotions('I am so excited!', 'emotions_27')\n",
        "print(f'Primary emotion: {result[\"primary_emotion\"]} ({result[\"primary_confidence\"]:.1f}%)')\n",
        "```\n\n",
        "## 📊 Usage Examples\n\n",
        "```python\n",
        "# Analyze different types of text\n",
        "texts = [\n",
        "    'I am absolutely thrilled about this partnership!',\n",
        "    'I feel betrayed and deeply disappointed.',\n",
        "    'This situation makes me really anxious.'\n",
        "]\n\n",
        "for text in texts:\n",
        "    result = emotion_system.detect_emotions(text, 'emotions_27', top_k=3)\n",
        "    if 'error' not in result:\n",
        "        primary = result['all_emotions'][0]\n",
        "        print(f'{text} → {primary[\"emotion\"]} ({primary[\"confidence\"]:.1f}%)')\n",
        "    else:\n",
        "        print(f'Error analyzing \"{text}\": {result[\"error\"]}')\n",
        "```\n\n",
        "## 📁 File Structure\n",
        "```\n",
        f"{model_dir}/\n",
        "├── models/                    # Saved model files\n",
        "│   ├── emotions_6/           # 6 basic emotions model\n",
        "│   ├── emotions_27/          # 27 detailed emotions model\n",
        "│   └── emotions_social/      # Social media emotions model\n",
        "├── config/\n",
        "│   └── model_config.json     # Model configuration\n",
        "├── emotion_system.pkl        # Complete system backup\n",
        "├── load_model.py            # Model loader\n",
        "├── api.py                   # FastAPI wrapper\n",
        "└── README.md               # This documentation\n",
        "```\n\n",
        "## 🎯 Use Cases\n\n",
        "- **Social Media Monitoring**: Track emotional sentiment\n",
        "- **Customer Support**: Analyze customer emotions\n",
        "- **Content Moderation**: Identify emotionally charged content\n",
        "- **Brand Monitoring**: Understand emotional response to campaigns\n",
        "- **Market Research**: Analyze emotional reactions\n\n",
        "## 📈 Performance\n\n",
        "- **Accuracy**: High accuracy across emotion categories\n",
        "- **Speed**: Fast inference for real-time applications\n",
        "- **Confidence Scoring**: Each emotion has confidence percentage\n",
        "- **Multi-Model**: Choose appropriate model based on use case\n\n",
        "## 🌐 API Usage\n\n",
        "Start the API server (from the model directory):\n",
        "```bash\n",
        "uvicorn api:app --reload\n",
        "```\n\n",
        "Use the API:\n",
        "```python\n",
        "import requests\n\n",
        "response = requests.post('http://localhost:8000/detect-emotions', json={\n",
        "    'text': 'I am so frustrated with this situation!',\n",
        "    'model_type': 'emotions_27',\n",
        "    'top_k': 5\n",
        "})\n\n",
        "result = response.json()\n",
        "print(f'Primary emotion: {result[\"primary_emotion\"]}')\n",
        "```\n\n",
        "---\n\n",
        "**Model Status**: ✅ Production Ready\n",
        "**Total Emotions**: 27+ different emotions\n",
        "**API Ready**: ✅ FastAPI wrapper included\n"
    ]

    readme_content = ''.join(readme_parts)

    with open(f"{model_dir}/README.md", 'w') as f:
        f.write(readme_content)

    # Create simple test script
    print("🧪 Creating test script...")
    test_code = """# --- EMOTION DETECTION TEST SCRIPT ---

from load_model import load_emotion_detection_model, quick_emotion_test

def comprehensive_emotion_test():
    \"\"\"
    Comprehensive test of emotion detection models
    \"\"\"
    print("🎭 COMPREHENSIVE EMOTION DETECTION TEST")
    print("=" * 50)

    # Load the model
    emotion_system, config = load_emotion_detection_model()

    if emotion_system is None:
        print("❌ Failed to load model")
        return

    # Test cases for different emotions
    test_cases = [
        ("Joy/Excitement", "I'm absolutely thrilled about this amazing opportunity!"),
        ("Anger/Frustration", "I'm so frustrated and angry about this situation!"),
        ("Fear/Anxiety", "I'm really worried and anxious about the outcome."),
        ("Sadness/Disappointment", "I feel deeply disappointed and sad about this."),
        ("Love/Gratitude", "I'm so grateful and I love working with this team!"),
        ("Surprise/Confusion", "I'm completely surprised and confused by this news."),
        ("Mixed Emotions", "I'm excited but also nervous about this big change.")
    ]

    print("📊 EMOTION DETECTION RESULTS:")
    print("-" * 60)

    for category, text in test_cases:
        result = emotion_system.detect_emotions(text, "emotions_27", top_k=5)

        if "error" not in result:
            primary = result['all_emotions'][0]
            print(f"\\n{category.upper()}:")
            print(f"Text: '{text}'")
            print(f"Primary: {primary['emoji']} {primary['emotion'].title()} ({primary['confidence']:.1f}%)")

            # Show top 3 emotions
            top_3_display = " | ".join([
                f"{e['emoji']} {e['emotion'].title()}"
                for e in result['all_emotions'][:3]
            ])
            print(f"Top 3: {top_3_display}")
        else:
            print(f"❌ Error analyzing '{text}': {result['error']}")

    print(f"\\n✅ Comprehensive testing completed!")
    print(f"🎭 Your emotion detection system can identify 27+ different emotions!")

if __name__ == "__main__":
    comprehensive_emotion_test()
"""

    with open(f"{model_dir}/test_emotions.py", 'w') as f:
        f.write(test_code)

    # Final verification
    print(f"\n✅ EMOTION DETECTION MODEL SAVED SUCCESSFULLY!")
    print(f"📍 Location: {model_dir}/")

    # Show what was saved
    print(f"\n📁 SAVED COMPONENTS:")
    def show_saved_files(directory, prefix=""):
        items = sorted(os.listdir(directory))
        for i, item in enumerate(items):
            item_path = os.path.join(directory, item)
            is_last = i == len(items) - 1
            current_prefix = "└── " if is_last else "├── "
            print(f"{prefix}{current_prefix}{item}")

            if os.path.isdir(item_path) and not item.startswith('.'):
                next_prefix = prefix + ("    " if is_last else "│   ")
                show_saved_files(item_path, next_prefix)

    show_saved_files(model_dir)

    print(f"\n🎉 READY TO USE!")
    print("To load your model in future sessions:")
    print("```python")
    print(f"from {model_dir}.load_model import load_emotion_detection_model")
    print("emotion_system, config = load_emotion_detection_model()")
    print("result = emotion_system.detect_emotions('Your text here!', 'emotions_27')")
    print("```")

    print(f"\n🧪 TO TEST:")
    print(f"```bash")
    print(f"python {model_dir}/test_emotions.py")
    print("```")

    return model_dir

# Execute the complete save
print("🚀 STARTING COMPLETE EMOTION MODEL SAVE...")
saved_model_path = save_emotion_detection_model_complete()

# Test the saved model
print(f"\n🧪 TESTING SAVED MODEL...")
# Use exec to run the loader script so its functions/classes are defined in the current scope
loader_script_path = os.path.join(saved_model_path, 'load_model.py')
if os.path.exists(loader_script_path):
    try:
        exec(open(loader_script_path).read())
        # Now call the functions defined in the executed script
        emotion_system, config = load_emotion_detection_model()

        if emotion_system:
            quick_emotion_test(emotion_system)

            print(f"\n🎭 SUCCESS! Your emotion detection model is saved and working!")
            # Calculate total emotions from the loaded config
            total_emotions = sum(len(m.get('emotions', [])) for m in config.get('models', {}).values())
            print(f"📊 Can detect {total_emotions} different emotions")
            print(f"🚀 Ready for production use!")
        else:
            print("❌ Error loading the saved model")
    except Exception as e:
        print(f"❌ Error executing test script: {e}")
else:
    print(f"❌ Loader script not found at {loader_script_path}")


print(f"\n💡 NEXT STEPS:")
print("1. Run comprehensive tests: `python emotion_detection_model_v1/test_emotions.py` (from the notebook terminal or your local terminal)")
print("2. Start API server: `cd emotion_detection_model_v1` then `uvicorn api:app --reload` (from the notebook terminal or your local terminal)")
print("3. Use in your code: `from emotion_detection_model_v1.load_model import load_emotion_detection_model`")

🚀 STARTING COMPLETE EMOTION MODEL SAVE...
💾 SAVING EMOTION DETECTION MODEL SYSTEM
📁 Created directory: emotion_detection_model_v1
🧠 Loading emotion detection models for saving...
📥 Loading 6 Basic Emotions...


Device set to use cpu


❌ Error saving 6 Basic Emotions: name 'model_dir' is not defined
📥 Loading 27 Detailed Emotions...


Device set to use cpu


❌ Error saving 27 Detailed Emotions: name 'model_dir' is not defined
📥 Loading Social Media Emotions...


Device set to use cpu


❌ Error saving Social Media Emotions: name 'model_dir' is not defined
🥒 Saving complete emotion system...
📝 Saving configuration...
🔧 Creating loader script...
🚀 Creating API wrapper...
📚 Creating documentation...
🧪 Creating test script...

✅ EMOTION DETECTION MODEL SAVED SUCCESSFULLY!
📍 Location: emotion_detection_model_v1/

📁 SAVED COMPONENTS:
├── README.md
├── api.py
├── config
│   └── model_config.json
├── emotion_system.pkl
├── load_model.py
├── models
│   ├── emotions_27
│   │   ├── config.json
│   │   ├── merges.txt
│   │   ├── model.safetensors
│   │   ├── special_tokens_map.json
│   │   ├── tokenizer.json
│   │   ├── tokenizer_config.json
│   │   └── vocab.json
│   ├── emotions_6
│   │   ├── config.json
│   │   ├── merges.txt
│   │   ├── model.safetensors
│   │   ├── special_tokens_map.json
│   │   ├── tokenizer.json
│   │   ├── tokenizer_config.json
│   │   └── vocab.json
│   └── emotions_social
│       ├── config.json
│       ├── merges.txt
│       ├── model.safetensors
│   

In [25]:
# --- TEST THE SAVED MODEL ---

def test_saved_emotion_model():
    """
    Test the saved emotion detection model
    """
    print("🧪 TESTING SAVED EMOTION MODEL")
    print("=" * 40)

    # Load the saved model
    exec(open(f'/content/drive/MyDrive/emotion_detection_model_v1/load_model.py').read())
    emotion_system, config = load_emotion_detection_model()

    if emotion_system is None:
        print("❌ Failed to load model")
        return

    # Test with various emotional texts
    test_cases = [
        ("Joy/Excitement", "I'm absolutely thrilled about this amazing opportunity!"),
        ("Anger/Frustration", "I'm so frustrated and angry about this situation!"),
        ("Fear/Anxiety", "I'm really worried and anxious about the outcome."),
        ("Sadness/Disappointment", "I feel deeply disappointed and sad about this."),
        ("Love/Gratitude", "I'm so grateful and I love working with this team!"),
        ("Surprise/Confusion", "I'm completely surprised and confused by this news."),
        ("Mixed Emotions", "I'm excited but also nervous about this big change.")
    ]

    print("📊 EMOTION DETECTION RESULTS:")
    print("-" * 50)

    for category, text in test_cases:
        result = emotion_system.detect_emotions(text, "emotions_27", top_k=3)

        if "error" not in result:
            primary = result['all_emotions'][0]
            print(f"\n{category.upper()}:")
            print(f"Text: '{text}'")
            print(f"Primary: {primary['emoji']} {primary['emotion'].title()} ({primary['confidence']:.1f}%)")

            # Show top 3 emotions
            top_3_display = " | ".join([
                f"{e['emoji']} {e['emotion'].title()}"
                for e in result['all_emotions'][:3]
            ])
            print(f"Top 3: {top_3_display}")
        else:
            print(f"❌ Error analyzing '{text}': {result['error']}")

    print(f"\n✅ Model testing completed successfully!")
    print(f"🎭 Your emotion detection system can identify 27+ different emotions!")

# Run the test
test_saved_emotion_model()


🧪 TESTING SAVED EMOTION MODEL
📥 Loading emotion detection model...
✅ Configuration loaded.
✅ System structure loaded.
🔄 Loading models from disk...
✅ Emotion detection model loaded successfully!
📊 Available models: 
📊 EMOTION DETECTION RESULTS:
--------------------------------------------------
❌ Error analyzing 'I'm absolutely thrilled about this amazing opportunity!': Model emotions_27 not available
❌ Error analyzing 'I'm so frustrated and angry about this situation!': Model emotions_27 not available
❌ Error analyzing 'I'm really worried and anxious about the outcome.': Model emotions_27 not available
❌ Error analyzing 'I feel deeply disappointed and sad about this.': Model emotions_27 not available
❌ Error analyzing 'I'm so grateful and I love working with this team!': Model emotions_27 not available
❌ Error analyzing 'I'm completely surprised and confused by this news.': Model emotions_27 not available
❌ Error analyzing 'I'm excited but also nervous about this big change.': Model e